# Content-based recommendation

# Exercise 1
Based on the TF-IDF vectors obtained in the Exercise 2 from Session 4, represent each user in the same vector space. Amongst other feasible solutions, you can represent a user (user profile) by computing the weighted mean of the items vectors. Compute the cosine similarity for user 'A39WWMBA0299ZF' and all products in the training set not rated by the user. What are the top-5 recommended items for user 'A39WWMBA0299ZF'? Print out the top-5 items and their similarity score.  

In [1]:
import pickle
import pandas as pd
import numpy as np

tf = pd.read_pickle("asin_tfidf3.pkl")

train = pd.read_pickle("train.pkl")
train = train[["overall", "reviewerID", "asin"]]

tf

,access,account,acroni,activ,address,adob,advanc,advantag,advisor,aftershot,...,290,291,292,293,294,295,296,297,298,299
asin,,,,,,,,,,,,,,,,,,,,,
0321700945,0.0,0.0,0.0,0.0,0.0,0.416422,0.0,0.0,0.0,0.000000,...,-0.139258,0.072900,0.011481,0.103516,0.216113,-0.011523,0.034961,-0.129614,-0.065100,0.170508
0321719816,0.0,0.0,0.0,0.0,0.0,0.287289,0.0,0.0,0.0,0.000000,...,-0.050659,0.103565,-0.028122,0.089251,0.060734,-0.083687,0.037777,-0.018997,-0.018507,0.021332
0321719824,0.0,0.0,0.0,0.0,0.0,0.269974,0.0,0.0,0.0,0.000000,...,-0.065192,0.004190,-0.059106,0.009880,-0.022641,-0.077301,0.045943,-0.010876,-0.039354,0.003113
0763855553,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,-0.057220,0.073804,-0.029626,0.005472,0.101263,-0.096191,-0.079675,-0.050153,-0.066357,0.033606
0982697813,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.020996,0.143896,-0.058105,0.039355,-0.010319,-0.082477,0.030371,-0.099390,0.006140,0.023193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B01F7RJHIQ,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,-0.094889,0.030273,-0.099270,0.024489,0.099799,-0.102675,-0.109165,-0.034155,-0.026706,0.000773
B01FFVDY9M,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.479046,...,-0.093079,0.056112,-0.017171,-0.008640,-0.003377,-0.112684,0.016276,-0.137783,-0.118788,0.095656
B01H39M7ME,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.120972,0.056885,-0.057129,-0.042175,0.193115,-0.083618,0.026932,-0.187500,0.054562,-0.008789


In [4]:
full = train.merge(tf, on="asin", how="left").dropna()
mask = ~full.columns.isin(["overall", "reviewerID", "asin"])
full.loc[:, mask]  = full.loc[:, mask].mul(full["overall"], axis=0)

mask2 = ~full.columns.isin(["overall", "asin"])
full = full.loc[:, mask2].groupby("reviewerID").mean()
# full.to_pickle("user_profiles.pkl")
full


# wm = lambda x: np.average(x, weights=df.loc[x.index, "overall"])
# full.drop("asin",axis=1).groupby("reviewerID").apply(lambda x: wm(x))
# weighted_average(full, ["asin", "overall"], "overall", "reviewerID")

,access,account,acroni,activ,address,adob,advanc,advantag,advisor,aftershot,...,290,291,292,293,294,295,296,297,298,299
reviewerID,,,,,,,,,,,,,,,,,,,,,
A100UD67AHFODS,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,-0.484395,0.176640,0.091954,0.109714,0.450016,-0.309309,-0.076063,-0.431012,-0.270338,0.175346
A105S56ODHGJEK,0.000000,0.035776,0.000000,0.0,0.0,0.0,0.0,0.0,0.090065,0.0,...,-0.308737,0.109594,0.004872,-0.097996,0.356185,-0.282419,-0.122896,-0.237245,-0.145221,0.210940
A1075X1Q4M3S78,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,-0.510763,-0.698051,0.042386,0.640352,0.732659,-0.548418,-0.543007,0.040734,-0.161328,0.274862
A10C5CJK1YKGV0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,-0.352237,0.178072,-0.012605,-0.128421,0.334304,-0.231970,-0.086603,-0.196387,-0.125653,0.284422
A10CRW7XRJBJ2G,0.000000,0.000000,0.116844,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,-0.162928,0.028587,-0.099716,-0.029412,0.349292,-0.382111,-0.252292,-0.244036,-0.007735,0.017381
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZD8SMNGQI98O,0.000000,0.178945,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,-0.060097,0.201214,0.084046,-0.152527,0.176422,-0.286704,-0.006282,-0.078435,-0.155033,0.280384
AZQGJ5CEAJGXB,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,-0.263689,0.221452,0.021396,-0.065379,-0.043327,-0.369486,-0.088769,-0.198493,-0.193061,0.213204
AZW10G02DNJI4,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,-0.247090,0.153326,-0.180114,0.100705,0.301591,-0.417238,-0.302458,-0.442512,-0.185656,0.152711


In [5]:
from sklearn.metrics.pairwise import cosine_similarity

u = "A100UD67AHFODS"

tff = tf.copy()
tff["sim"] =  cosine_similarity(tff, full[full.index == u])
# tff.drop("asin",axis=1) @ full[full.index == u].T / (np.linalg.norm(tff.drop("asin",axis=1))*np.linalg.norm(full[full.index == u].T))
tff["sim"].nlargest(5)
# 'B019FWRG3C', 0.419),
# ('B00W259T7G', 0.206),
# ('B00IJHY54S', 0.102),
# ('B0006O10P4', 0.094),
# ('B000LIBUBY', 0.083)]


asin
B005S4Y65I    0.910430
B009CCVMO0    0.909132
B009HBCZPQ    0.906524
B00440D8PG    0.903363
B001GL8USQ    0.880053
Name: sim, dtype: float64

In [6]:
df_test = pd.read_pickle("testset.pkl")[["reviewerID", "asin", "overall"]]
df_test = df_test.rename(columns={"reviewerID": "uid", "asin": "iid"})
len(df_test)

1711

In [7]:
# pred_list = []

# for u in set(df_test["uid"]): 
#     items = tf.copy()
#     items["sim"] = euclidean_distances(items.loc[:,items.columns[:646]], full[full.index == u].loc[:,items.columns[:646]]) + \
#                      euclidean_distances(items.loc[:,items.columns[646:]], full[full.index == u].loc[:,items.columns[646:]])
#     items["sim"] /= 2
#     items = items.drop(train[train["reviewerID"]==u].asin)
    
#     user_preds = pd.DataFrame(items["sim"].reset_index())
#     user_preds["uid"] = u 
#     pred_list.append(user_preds)
    
# preds = pd.concat(pred_list).reset_index(drop=True)
# preds = preds.rename(columns={"asin": "iid", "sim": "score"})
# preds

In [8]:
from sklearn.metrics.pairwise import cosine_similarity

pred_list = []

for u in set(df_test["uid"]): 
    items = tf.copy()
    items["sim"] = cosine_similarity(items, full[full.index == u])
    items = items.drop(train[train["reviewerID"]==u].asin)
    
    user_preds = pd.DataFrame(items["sim"].reset_index())
    user_preds["uid"] = u 
    pred_list.append(user_preds)
    
preds = pd.concat(pred_list).reset_index(drop=True)
preds = preds.rename(columns={"asin": "iid", "sim": "score"})
preds

,iid,score,uid
0,0321700945,0.581499,A14V7Y1TE21VT3
1,0321719816,0.503276,A14V7Y1TE21VT3
2,0321719824,0.423287,A14V7Y1TE21VT3
3,0763855553,0.668405,A14V7Y1TE21VT3
4,0982697813,0.464798,A14V7Y1TE21VT3
...,...,...,...
1360952,B01F7RJHIQ,0.503962,A1RMEDDRG0V9F2
1360953,B01FFVDY9M,0.512868,A1RMEDDRG0V9F2
1360954,B01H39M7ME,0.340230,A1RMEDDRG0V9F2
1360955,B01HAP47PQ,0.412546,A1RMEDDRG0V9F2


In [9]:
items = tf.copy()
items["sim"] = cosine_similarity(items, full[full.index == u])
items = items.drop(train[train["reviewerID"]==u].asin)
items.nlargest(5, "sim")

,access,account,acroni,activ,address,adob,advanc,advantag,advisor,aftershot,...,291,292,293,294,295,296,297,298,299,sim
asin,,,,,,,,,,,,,,,,,,,,,
B01326J62G,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.030197,0.007645,-0.021805,0.062469,-0.089867,0.013191,-0.023571,-0.044434,0.035461,0.902963
B00M76N6MO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.004532,0.012589,-0.024185,0.073090,-0.093285,0.036507,-0.051922,-0.081299,0.050476,0.890287
B00M9GTEPA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.015720,-0.008277,-0.025374,0.035970,-0.049767,0.027934,-0.061289,-0.054525,0.022298,0.831352
B000GI0HR2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.007141,0.011983,-0.015299,0.073283,-0.136719,0.036591,-0.064723,-0.073201,0.070394,0.814559
B00M9GTL6M,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.027797,-0.027954,-0.023019,0.036621,-0.075849,0.085449,-0.062810,-0.051095,-0.000523,0.801797


In [10]:

# TEST
df_test = pd.read_pickle("testset.pkl")[["reviewerID", "asin", "overall"]]
df_test = df_test.rename(columns={"reviewerID": "uid", "asin": "iid"})

# PREDICTIONS
nb = preds
pred_nb_list = list(nb.itertuples(index=False))


# Detect users from training set that are not in test
nb_users = set([pred.uid for pred in pred_nb_list])
nb_users_in_pred_but_not_in_test = list(nb_users.difference(set(df_test['uid'])))
print(f"There are {len(nb_users_in_pred_but_not_in_test)} users in the training set that are not in the test set.")

# Remove these users' predictions for evaluation
print("Lengths before removing preds not in test set:", len(nb))
nb = nb[~nb.uid.isin(nb_users_in_pred_but_not_in_test)]
print("After removing:", len(nb))

nb_merge = nb.merge(df_test, how="inner", on=["uid", "iid"])
print("\nkNN RMSE:", np.sqrt(np.mean((nb_merge["overall"] - nb_merge["score"]*5)**2)))


There are 0 users in the training set that are not in the test set.
Lengths before removing preds not in test set: 1360957
After removing: 1360957

kNN RMSE: 2.0416003657891015


# Exercise 2



Compute the systems’ hit rate based on the top-5, top-10 and top-20 recommendations, averaged over the total number of users. Remember that, as we are evaluating the system, you should compute the hit rate over the test set. How well/bad does this Content-based approach perform compared to the Collaborative Filtering?

In [11]:
def relevant_column(preds, df_test, k): 
    topKpreds = preds.groupby(['uid']).apply(lambda x: x.nlargest(k,['score'])).reset_index(drop=True)[["uid", "iid", "score"]]
    merged = topKpreds.merge(df_test[["uid", "iid", "overall"]], how="left", on=["uid", "iid"])
    merged["relevant"] = (merged["overall"] >= 4) * 1 
    return merged

def PatK(preds, df_test, k):
    merged = relevant_column(preds, df_test, k)
    score  = merged[["uid", "iid", "relevant"]].groupby(by="uid")["relevant"].mean().mean()
    return score

def first(x):
    for i in range(len(x)):
        if x.iloc[i].relevant == 1:
            return 1/(i+1)
    return 0 
def MRRatK(preds, df_test, k):
    merged = relevant_column(preds, df_test, k)
    score  = merged[["uid", "iid", "relevant"]].groupby(by="uid").apply(first).mean()
    return score

def MAPatK(preds, df_test, k):
    merged = relevant_column(preds, df_test, k)
    score  = merged[["uid", "iid", "relevant"]].groupby(by="uid")["relevant"].apply(lambda x: 1./np.arange(1,k+1) @ x).mean()
    return score

def HRatK(preds, df_test, k):
    merged = relevant_column(preds, df_test, k)
    score = merged[["uid", "iid", "relevant"]].groupby(by="uid")["relevant"].apply(lambda x: x.any()*1).mean()
    return score

In [12]:
ks = [5, 10, 20]


for k in ks:
    P   = PatK(preds, df_test, k)  
    MAP = MAPatK(preds, df_test, k)
    MRR = MRRatK(preds, df_test, k)
    HR = HRatK(preds, df_test, k)
    print(f"  P@{k:2g} = {P  :.4f}")
    print(f"MAP@{k:2g} = {MAP:.4f}")
    print(f"MRR@{k:2g} = {MRR:.4f}")
    print(f" HR@{k:2g} = {HR :.4f}\n")


  P@ 5 = 0.0312
MAP@ 5 = 0.0866
MRR@ 5 = 0.0866
 HR@ 5 = 0.1560

  P@10 = 0.0195
MAP@10 = 0.0918
MRR@10 = 0.0918
 HR@10 = 0.1952

  P@20 = 0.0130
MAP@20 = 0.0962
MRR@20 = 0.0962
 HR@20 = 0.2595



In [19]:
preds

,iid,score,uid
0,0321700945,0.581499,A14V7Y1TE21VT3
1,0321719816,0.503276,A14V7Y1TE21VT3
2,0321719824,0.423287,A14V7Y1TE21VT3
3,0763855553,0.668405,A14V7Y1TE21VT3
4,0982697813,0.464798,A14V7Y1TE21VT3
...,...,...,...
1360952,B01F7RJHIQ,0.503962,A1RMEDDRG0V9F2
1360953,B01FFVDY9M,0.512868,A1RMEDDRG0V9F2
1360954,B01H39M7ME,0.340230,A1RMEDDRG0V9F2
1360955,B01HAP47PQ,0.412546,A1RMEDDRG0V9F2


In [17]:
# preds.to_pickle("preds_content.pkl")

In [13]:
# on TITLE only + 3 factors
#   P@ 5 = 0.0312
# MAP@ 5 = 0.0866
# MRR@ 5 = 0.0866
#  HR@ 5 = 0.1560

#   P@10 = 0.0195
# MAP@10 = 0.0918
# MRR@10 = 0.0918
#  HR@10 = 0.1952

#   P@20 = 0.0130
# MAP@20 = 0.0962
# MRR@20 = 0.0962
#  HR@20 = 0.2595

# 4
#   P@ 5 = 0.0321
# MAP@ 5 = 0.1038
# MRR@ 5 = 0.1038
#  HR@ 5 = 0.1607

#   P@10 = 0.0206
# MAP@10 = 0.1098
# MRR@10 = 0.1098
#  HR@10 = 0.2063

#   P@20 = 0.0139
# MAP@20 = 0.1146
# MRR@20 = 0.1146
#  HR@20 = 0.2770

HRatK(nb, df_test, 5), HRatK(nb, df_test, 10), HRatK(nb, df_test, 20)
#(0.14436002337814144, 0.1952074810052601, 0.25306838106370544)


(0.15604909409701928, 0.1952074810052601, 0.25949736995908823)

In [14]:
asd

NameError: name 'asd' is not defined

# Exercise 3

Repeat Exercise 1 and 2, this time representing the products and users in a word2vec vector space. You may use the gensim library and download the 300-dimension embeddings from Google. Source: https://radimrehurek.com/gensim/models/word2vec.html#pretrained-models